## Selenium on footballDB

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

import urllib
import requests

In [8]:
 class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
        
        
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [84]:
def get_player_table(url, cat):
    hp = HTMLTableParser()
    table = hp.parse_url(url)[0][1] # Grabbing the table from the tuple

    # Fix up headers
    cols = list(table.columns)
    new_cols = [col.strip('\n') for col in cols]
    table.columns = new_cols

    table['Name'] = table['Name'].str.replace('\n', '').str.replace(cat, '').str.strip()
    return table.copy()

### Power Up Players

In [26]:
df_list = []

for page_num in range(1,8):   
    url = f'https://www.muthead.com/19/players?filter-market=3&filter-program-19=229&page={page_num}'
    df_list.append(get_player_table(url, 'Power Up'))
    time.sleep(2)
        
mut_data = pd.concat(df_list)

In [74]:
mut_data.head()

,,Name,OVR,POS,PS4
0,,Eddie George,74.0,HB,"1,900"
1,,Kevin Greene,74.0,LOLB,"1,700"
2,,Donovan McNabb,74.0,QB,"1,900"
3,,Jonathan Ogden,74.0,LT,"1,900"
4,,Deion Sanders,74.0,CB,"8,400"


### Hall Of Fame Players

In [72]:
url2 = 'https://www.muthead.com/19/players?filter-program-19=238&filter-market=3&&filter-tier=8'
hof_df = get_player_table(url2, 'Hall of Fame')

In [73]:
hof_df

,,Name,OVR,POS,PS4
0,,Terrell Davis,77.0,HB,14.0K
1,,Dan Marino,77.0,QB,13.0K
2,,Terrell Owens,77.0,WR,13.8K
3,,Brian Urlacher,77.0,MLB,15.0K
4,,Rod Woodson,77.0,CB,13.2K


### Master Players

In [75]:
url3 = 'https://www.muthead.com/19/players?filter-program-19=236&filter-market=3&'
master_df = get_player_table(url3, 'Master')

In [76]:
master_df

,,Name,OVR,POS,PS4
0,,Ryan Shazier,70.0,MLB,N/A
1,,Ricky Williams,70.0,HB,N/A


### Team Captains

In [85]:
url4 = 'https://www.muthead.com/19/players?filter-program-19=244&filter-market=3&'
team_captains = get_player_table(url4, 'Team Captain')

In [86]:
team_captains

,,Name,OVR,POS,PS4
0,,Ray Lewis,75.0,MLB,N/A
1,,Jerry Rice,75.0,WR,N/A
2,,Shannon Sharpe,75.0,TE,N/A
3,,Michael Strahan,75.0,LE,N/A


In [88]:
mut_cats = [mut_data, hof_df, master_df, team_captains]
all_players = pd.concat(mut_cats).reset_index(drop=True)

In [89]:
all_players.shape

(184, 5)

In [90]:
#mut_data.to_csv('mut_data_sep29.csv')
#all_players.to_csv('mut_data2_sep29.csv')

In [91]:
all_players.head(10)

,,Name,OVR,POS,PS4
0,,Eddie George,74.0,HB,"1,900"
1,,Kevin Greene,74.0,LOLB,"1,700"
2,,Donovan McNabb,74.0,QB,"1,900"
3,,Jonathan Ogden,74.0,LT,"1,900"
4,,Deion Sanders,74.0,CB,"8,400"
5,,Lawrence Taylor,74.0,ROLB,"2,100"
6,,Larry Allen,73.0,RG,"2,300"
7,,Champ Bailey,73.0,CB,"2,500"
8,,Tim Brown,73.0,WR,"1,900"
9,,Brian Dawkins,73.0,FS,"2,700"


In [32]:
august_df = pd.read_csv('/Users/mattjohnson/Desktop/Python2018/MUT/MUT19-PowerUp-Team-Analysis/csv/powerup_aug18.csv')
august_df.drop('Unnamed: 0', axis=1, inplace=True)

In [59]:
august_df = august_df[:-2]
august_df.tail(10)

,Name,OVR,POS,PS4,firstName,lastName,POS2,Player,Pos,College,...,Phi,Pit,SF,Sea,TB,Ten,Was,RAMS,CHARGERS,numTeams
156,Dan Marino,0.0,QB,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
157,Terrell Owens,0.0,WR,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
158,Brian Urlacher,0.0,MLB,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
159,Rod Woodson,0.0,CB,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
160,Demarcus Ware,0.0,ROLB,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
161,Tim Brown,0.0,WR,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
162,Ray Lewis,0.0,MLB,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
163,Jerry Rice,0.0,WR,0,0,0,0,0,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
164,Shannon Sharpe,0.0,TE,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
165,Michael Strahan,0.0,LE,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [92]:
aug_names = set(august_df['Name'].values)
sep_names = set(all_players['Name'].values)

inter = aug_names.intersection(sep_names)
d1 = aug_names.difference(sep_names)
d2 = sep_names.difference(aug_names)
len(aug_names)
print(len(d1), len(d2), len(inter))

1 19 165


In [93]:
d1 # 

{'Demarcus Ware'}

In [94]:
d2 # unfinished 

{'Aeneas Williams',
 'Barry Sanders',
 'Brian Dawkins',
 'DeMarcus Ware',
 'Deion Sanders',
 'Derek Carr',
 'Donovan McNabb',
 "Dont'a Hightower",
 'Eddie George',
 'Ha Ha Clinton-Dix',
 'John Randle',
 'Jonathan Ogden',
 'Kevin Greene',
 'Larry Allen',
 'Lawrence Taylor',
 'Patrick Mahomes II',
 'Ricky Williams',
 'Ryan Shazier',
 'Tony Gonzalez'}

In [101]:
for _ in d2:
    print(_)

Brian Dawkins
Patrick Mahomes II
Tony Gonzalez
Larry Allen
Dont'a Hightower
John Randle
Lawrence Taylor
Barry Sanders
DeMarcus Ware
Ha Ha Clinton-Dix
Eddie George
Deion Sanders
Jonathan Ogden
Ryan Shazier
Kevin Greene
Aeneas Williams
Donovan McNabb
Ricky Williams
Derek Carr


In [95]:
#powerups = pd.read_csv('madden19_powerups.csv')
#cols = powerups.columns.drop('Unnamed: 4')
#powerups.drop('Name', axis=1, inplace=True)
#powerups.columns = cols

powerups = all_players.copy()

#powerups['Name'] = powerups['Name'].str[:-8]

df2 = pd.DataFrame(powerups['Name'].str.split())
powerups.loc[:, 'firstName'] = df2.Name.map(lambda x: x[0])
powerups.loc[:, 'lastName'] = df2.Name.map(lambda x: ' '.join(x[1:]))

powerups['POS'].value_counts()

position_dict = {'CB':'DB', 'SS':'DB', 'FS':'DB', 'WR':'WR', 'QB':'QB', 'HB':'RB', 'FB':'RB', 
        'TE':'TE', 'LE':'DE', 'RE':'DE', 'DT':'DT', 'LOLB':'LB', 'ROLB':'LB', 'MLB':'LB', 
        'LG':'OG', 'RG':'OG', 'LT':'OT', 'RT':'OT', 'C':'C'}

powerups['POS2'] = powerups['POS'].map(position_dict)

In [104]:
unfins = list(powerups[ powerups['Name'].isin(d2)].index)

In [96]:
powerups.head()

,,Name,OVR,POS,PS4,firstName,lastName,POS2
0,,Eddie George,74.0,HB,"1,900",Eddie,George,RB
1,,Kevin Greene,74.0,LOLB,"1,700",Kevin,Greene,LB
2,,Donovan McNabb,74.0,QB,"1,900",Donovan,McNabb,QB
3,,Jonathan Ogden,74.0,LT,"1,900",Jonathan,Ogden,OT
4,,Deion Sanders,74.0,CB,"8,400",Deion,Sanders,DB


In [97]:
powerups.tail()

,,Name,OVR,POS,PS4,firstName,lastName,POS2
179,,Ricky Williams,70.0,HB,N/A,Ricky,Williams,RB
180,,Ray Lewis,75.0,MLB,N/A,Ray,Lewis,LB
181,,Jerry Rice,75.0,WR,N/A,Jerry,Rice,WR
182,,Shannon Sharpe,75.0,TE,N/A,Shannon,Sharpe,TE
183,,Michael Strahan,75.0,LE,N/A,Michael,Strahan,DE


In [98]:
print(powerups.shape)

(184, 8)


In [65]:
def get_player_teams(first_name, last_name, playerPos, driver):

    try:
        # Clearing the search field text box
        driver.find_element_by_xpath( \
            '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[1]').clear()

        # Sending the last name of current player
        driver.find_element_by_xpath( \
            '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[1]').send_keys(last_name)

        # Clicking the submit button to search
        driver.find_element_by_xpath( \
        '/html/body/div[1]/div[1]/div[1]/div[3]/div[5]/form/input[2]').click()
        time.sleep(2)
    
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        tables = soup.find_all('table')

        df = pd.read_html(str(tables[0]))[0]

        player = df[ df['Player']==(last_name+', '+first_name)]

        if len(player)>1:
            player = player[ player['Pos']==playerPos]
        
        return player
    except:
        print("# ERROR:")
        return []

In [106]:
driver = webdriver.Firefox()

driver.get('https://www.footballdb.com/players/players.html?q=develin')

In [111]:
player_list = []

for player in unfins:
    current = powerups.iloc[player]
    first, last, pos = current['firstName'], current['lastName'], current['POS2']
    
    try:
        player_list.append(get_player_teams(first, last, pos, driver))
    except:
        print('# ERROR #')
    time.sleep(1)

In [112]:
player_list_df = pd.concat(player_list)
#player_list.to_csv('player_list.csv')

In [114]:
player_list_a = pd.read_csv('/Users/mattjohnson/Desktop/Python2018/MUT/player_list.csv')

In [118]:
#player_list_a.drop('Unnamed: 0', axis=1, inplace=True)
print(player_list_a.shape)
player_list_a.head()

(122, 4)


,Player,Pos,College,Teams
0,"Hendricks, Ted",LB,Miami,"1969-1973 Bal, 1974 GB, 1975-1981 Oak, 1982-19..."
1,"Roaf, Willie",OT,Louisiana Tech,"1993-2001 NO, 2002-2005 KC"
2,"Smith, Bruce",DE,Virginia Tech,"1985-1999 Buf, 2000-2003 Was"
3,"Taylor, Sean",DB,Miami,2004-2007 Was
4,"Adams, Jamal",DB,Louisiana State,2017-2018 NYJ


In [120]:
player_list_df.shape

(15, 4)

In [122]:
all_data = pd.concat([player_list_a, player_list_df])

In [ ]:
list(all_data[''])

In [123]:
all_data

,Player,Pos,College,Teams
0,"Hendricks, Ted",LB,Miami,"1969-1973 Bal, 1974 GB, 1975-1981 Oak, 1982-19..."
1,"Roaf, Willie",OT,Louisiana Tech,"1993-2001 NO, 2002-2005 KC"
2,"Smith, Bruce",DE,Virginia Tech,"1985-1999 Buf, 2000-2003 Was"
3,"Taylor, Sean",DB,Miami,2004-2007 Was
4,"Adams, Jamal",DB,Louisiana State,2017-2018 NYJ
5,"Amos, Adrian",DB,Penn State,2015-2018 Chi
6,"Atkins, Geno",DT,Georgia,2010-2018 Cin
7,"Barr, Anthony",LB,UCLA,2014-2018 Min
8,"Berry, Eric",DB,Tennessee,2010-2018 KC
9,"Bosa, Joey",DE,Ohio State,"2016 SD, 2017-2018 LAC"


In [184]:
all_data['Player'] = all_data['lastName'] + ', '+ all_data['firstName']

merged_df = pd.merge(all_data, player_list, on='Player', how='outer')

In [190]:
fins = all_data[ all_data['empty']==0].index

driver = webdriver.Firefox()

driver.get('https://www.footballdb.com/players/players.html?q=develin')

In [108]:
player_list2 = []

for player in unfins:
    current = all_data.iloc[player]
    first, last, pos = current['firstName'], current['lastName'], current['POS2']
    
    try:
        player_list2.append(get_player_teams(first, last, pos, driver))
    except:
        print('# ERROR #')
    time.sleep(1)

NameError: name 'all_data' is not defined

In [ ]:
players2 = pd.concat(player_list2)
player_df = pd.concat([players2.reset_index(drop=True), player_list.reset_index(drop=True)])

In [205]:
player_df.shape

(141, 4)

In [ ]:
powerups['Player'] = powerups['lastName'] + ', '+ powerups['firstName']

In [212]:
all_players = set(powerups['Player'].unique())
some_players = set(player_df['Player'].unique())

In [213]:
C = all_players.difference(some_players)

In [235]:
merged = pd.merge(powerups, player_df, on='Player', how='outer')
merged['Teams'] = merged['Teams'].fillna(' ')
merged['Teams'] = merged['Teams'].str.replace('[^a-zA-Z,]', '')

In [285]:
df = merged.Teams.str.split('\s*,\s*', expand=True)\
                .apply(pd.Series.value_counts, 1).iloc[:, 1:].fillna(0)
    
df['LAR'] = df['LA'] + df['LAR'] 
df['RAMS'] = np.where(df['LAR']+df['Stl']>=1, 1, 0)

df['CHARGERS'] = np.where(df['LAC']+df['SD']>=1, 1, 0)

df.drop(['LA', 'NFLEHam', 'Stl', 'LAC', 'LAR', 'SD'], axis=1, inplace=True)

In [286]:
all_df = pd.concat([merged, df], axis=1)
#all_df.to_csv('all_powerups.csv')

In [284]:
remaining = ['Beckham Jr, Odell', 'Brown, Zach', 'Gurley II, Todd', 'Harris Jr, Chris',
 'Harrison Sr, Damon', 'Johnson, Lane', 'Johnson, Trumaine', 'Jones, Reshad', 'Slay Jr, Darius',
 'Smith, Jimmy', 'Smith, Telvin', 'Smith, Tyron', 'Tate III, Golden', 'Thomas III, Earl',
 'Williams, Trent']

rem_teams = [['NYG'], ['Ten', 'Buf', 'Was'], ['RAMS'], ['Den'], ['NYJ', 'NYG'], ['Phi'],
            ['RAMS', 'NYJ'], ['Mia'], ['Det'], ['Bal'], ['Jax'], ['Dal'], ['Sea', 'Det'], 
            ['Sea'], ['Was']]

In [288]:
all_df.to_csv('aug17_final_d')

,Name,OVR,POS,PS4,firstName,lastName,POS2,Player,Pos,College,...,Oak,Phi,Pit,SF,Sea,TB,Ten,Was,RAMS,CHARGERS
0,Champ Bailey,73,CB,"2,000",Champ,Bailey,DB,"Bailey, Champ",DB,Georgia,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
1,Randy Moss,73,WR,"3,400",Randy,Moss,WR,"Moss, Randy",WR,Marshall,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0,0
2,Michael Vick,73,QB,"4,100",Michael,Vick,QB,"Vick, Michael",QB,Virginia Tech,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0
3,Reggie White,73,LE,"1,900",Reggie,White,DE,"White, Reggie",DE,Tennessee,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,Derrick Brooks,72,ROLB,"2,800",Derrick,Brooks,LB,"Brooks, Derrick",LB,Florida State,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
